In [8]:
import numpy as np # linear algebra
import spacy
nlp = spacy.load('en_core_web_sm')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import base64
import string
import re
from collections import Counter
from time import time
from nltk.corpus import stopwords
import nltk
import heapq

%matplotlib inline

nltk.download('stopwords')
stopwords = stopwords.words('english')
sns.set_context('notebook')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [9]:
import os
os.makedirs("./../homework_6", exist_ok=True)

In [10]:
reviews = pd.read_csv("./../homework_6/data/wine_reviews.csv", usecols =['title', 'description', 'points', 'price', 'variety'], encoding='latin1')
reviews = reviews.dropna()
reviews.head(15)

,description,points,price,title,variety
1,"This is ripe and fruity, a wine that is smooth...",87,15.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red
2,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris
3,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling
4,"Much like the regular bottling from 2012, this...",87,65.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir
5,Blackberry and raspberry aromas show a typical...,87,15.0,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot
6,"Here's a bright, informal red that opens with ...",87,16.0,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato
7,This dry and restrained wine offers spice in p...,87,24.0,Trimbach 2012 Gewurztraminer (Alsace),GewÃ¼rztraminer
8,Savory dried thyme notes accent sunnier flavor...,87,12.0,Heinz Eifel 2013 Shine GewÃ¼rztraminer (Rheinh...,GewÃ¼rztraminer
9,This has great depth of flavor with its fresh ...,87,27.0,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris
10,"Soft, supple plum envelopes an oaky structure ...",87,19.0,Kirkland Signature 2011 Mountain CuvÃ©e Cabern...,Cabernet Sauvignon


In [11]:

import os
import pandas as pd


def read_csv(filepath: str, dropna: bool = True, dropduplicates: bool =True, encoding: str = 'latin1') -> pd.DataFrame:
    """
    read_csv returns DataFrame read from specified CSV format file.

    :param filepath: A path to file with data.
    :param dropna: Flag specifying if missed values should be deleted or not.
    :param encoding: Encoding to use for UTF when reading.
    :returns: A Pandas DataFrame.

    Example usage:
        read_csv(filepath='./data.csv', dropna=False)
    """
    try:
        data = pd.read_csv(filepath, encoding=encoding)
        if dropna:
            data = data.dropna()
        if dropduplicates:
            data = data.drop_duplicates()
        return data
    except FileNotFoundError:
        print(f"File not found: {filepath}")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
    

In [12]:
reviews = reviews.drop_duplicates()
reviews = reviews.reset_index()

In [15]:
!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/587.7 MB 2.6 MB/s eta 0:03:50
     ---------------------------------------- 0.2/587.7 MB 2.6 MB/s eta 0:03:47
     ---------------------------------------- 0.3/587.7 MB 2.4 MB/s eta 0:04:03
     ---------------------------------------- 0.4/587.7 MB 2.4 MB/s eta 0:04:04
     ---------------------------------------- 0.5/587.7 MB 2.4 MB/s eta 0:04:04
     ---------------------------------------- 0.6/587.7 MB 2.2 MB/s eta 0:04:28
     ---------------------------------------- 0.7/587.7 MB 2.3 MB/s eta 0:04:15
     ---------------------------------------- 0.8/587.7 MB 2.3 MB/s eta 0:04:17
     ---------------------------------------- 0.9/587.7 MB 2.3 MB/s eta 0:04:15
     ---------------------------------------- 1.0/587.7 MB 2.2 MB/s eta 0:04:23
     ---------------------------------------- 1.1/587.7 MB 2.3 MB/s eta 0:04:15
     ---------------------------------------- 1

In [16]:
nlp = spacy.load('en_core_web_lg')
def normalize_text(text):
    text = text.lower()
    tm1 = re.sub('<pre>.*?</pre>', '', text, flags=re.DOTALL)
    tm2 = re.sub('<code>.*?</code>', '', tm1, flags=re.DOTALL)
    tm3 = re.sub('<[^>]+>©', '', tm1, flags=re.DOTALL)
    return tm3.replace("\n", "")

In [17]:

reviews['description_Cleaned_1'] = reviews['description'].apply(normalize_text)

In [18]:
print('Before normalizing text-----\n')
print(reviews['description'][2])
print('\nAfter normalizing text-----\n')
print(reviews['description_Cleaned_1'][2])

Before normalizing text-----

Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.

After normalizing text-----

pineapple rind, lemon pith and orange blossom start off the aromas. the palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.


In [19]:
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~©'
# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    doc = nlp(docs, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return pd.Series(texts)

reviews['Description_Cleaned'] = reviews['description_Cleaned_1'].apply(lambda x: cleanup_text(x, False))

In [21]:

from collections import Counter

def generate_summaries(df, input_column, output_column, num_sentences=3):
    # Define a function to summarize text

    def calculate_sentence_importance(sentence):
    # Tokenize the sentence into words
      words = sentence.split()

    # Calculate word frequencies using Counter
      word_freq = Counter(words)

    # Calculate the sentence importance as the sum of word frequencies
      sentence_importance = sum(word_freq.values())

      return sentence_importance

    def summarize_text(text):
        doc = nlp(text)

        # Tokenize the text into sentences
        sentences = [sent.text for sent in doc.sents]

        # Calculate sentence importance scores
        sentence_scores = [calculate_sentence_importance(sent) for sent in sentences]

        # Rank sentences by importance and select the top 'num_sentences' sentences
        ranked_sentences = sorted(
            ((sentence, score) for sentence, score in zip(sentences, sentence_scores)),
            key=lambda x: x[1],
            reverse=True)[:num_sentences]

        # Generate the summarized text
        summarized_text = [sentence for sentence, _ in ranked_sentences]

        return ' '.join(summarized_text)

    # Apply the summarization function to the input_column
    df[output_column] = df[input_column].apply(summarize_text)

    return df

generate_summaries(reviews, 'Description_Cleaned', 'summary')


,index,description,points,price,title,variety,description_Cleaned_1,Description_Cleaned,summary
0,1,"This is ripe and fruity, a wine that is smooth...",87,15.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,"this is ripe and fruity, a wine that is smooth...",ripe fruity wine smooth still structure . firm...,firm tannin fill juicy red berry fruit freshen...
1,2,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,"tart and snappy, the flavors of lime flesh and...",tart snappy flavor lime flesh rind dominate . ...,tart snappy flavor lime flesh rind dominate . ...
2,3,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,"pineapple rind, lemon pith and orange blossom ...",pineapple rind lemon pith orange blossom start...,palate bit opulent note honey drizzle guava ma...
3,4,"Much like the regular bottling from 2012, this...",87,65.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,"much like the regular bottling from 2012, this...",much like regular bottling 2012 come across ra...,much like regular bottling 2012 come across ra...
4,5,Blackberry and raspberry aromas show a typical...,87,15.0,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,blackberry and raspberry aromas show a typical...,blackberry raspberry aroma show typical navarr...,blackberry raspberry aroma show typical navarr...
...,...,...,...,...,...,...,...,...,...
111587,129966,Notes of honeysuckle and cantaloupe sweeten th...,90,28.0,Dr. H. Thanisch (Erben MÃ¼ller-Burggraef) 2013...,Riesling,notes of honeysuckle and cantaloupe sweeten th...,note honeysuckle cantaloupe sweeten deliciousl...,intensely juicy quench palate stream tart tang...
111588,129967,Citation is given as much as a decade of bottl...,90,75.0,Citation 2004 Pinot Noir (Oregon),Pinot Noir,citation is given as much as a decade of bottl...,citation give much decade bottle age prior rel...,citation give much decade bottle age prior rel...
111589,129968,Well-drained gravel soil gives this wine its c...,90,30.0,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,GewÃ¼rztraminer,well-drained gravel soil gives this wine its c...,well drain gravel soil give wine crisp dry cha...,well drain gravel soil give wine crisp dry cha...
111590,129969,"A dry style of Pinot Gris, this is crisp with ...",90,32.0,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,"a dry style of pinot gris, this is crisp with ...",dry style pinot gris crisp acidity . also weig...,also weight solid powerful core spice baked ap...


In [23]:
print('Original text-----\n')
print(reviews['description'][2])
print('\nAfter summeryzation-----\n')
print(reviews['summary'][2])

Original text-----

Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.

After summeryzation-----

palate bit opulent note honey drizzle guava mango give way slightly astringent semidry finish . pineapple rind lemon pith orange blossom start aroma .


In [24]:

from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize

def create_tagged_documents(dataframe, text_column='summary'):
    """
    Create tagged documents for Doc2Vec model from a DataFrame.

    Args:
        dataframe (pd.DataFrame): Input DataFrame containing text data.
        text_column (str): The name of the column containing the text data.

    Returns:
        list of gensim.models.doc2vec.TaggedDocument: List of tagged documents.
    """
    data = dataframe[text_column].astype(str).tolist()

    tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
    
    return tagged_data


In [26]:


from gensim.models.doc2vec import Doc2Vec

def train_doc2vec_model(tagged_data, vec_size=100, alpha=0.05, max_epochs=10):
    """
    Train a Doc2Vec model using the provided tagged data and parameters.

    Args:
        tagged_data (list of gensim.models.doc2vec.TaggedDocument): List of tagged documents.
        vec_size (int): The size of the vector representation for documents.
        alpha (float): The initial learning rate for training.
        max_epochs (int): The maximum number of training epochs.

    Returns:
        gensim.models.doc2vec.Doc2Vec: Trained Doc2Vec model.
    """
    model = Doc2Vec(vector_size=vec_size,
                    alpha=alpha,
                    min_alpha=0.00025,
                    min_count=1,
                    dm=1)

    model.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.epochs)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha
    
    return model


In [27]:
tagged_data = create_tagged_documents(reviews)

In [28]:
model = train_doc2vec_model(tagged_data)

iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9


In [29]:
test_data = word_tokenize("build 150 year six generation winemaking tradition winery trend toward lean style classic california buttercream aroma cut tart green apple .".lower())

In [ ]:
model.save("d2v.model")
print("Model Saved")

Model Saved


In [30]:

def get_most_similar_docs(test_data, model):
   # Infer the vector for the test document
   inferred_vector = model.infer_vector(test_data)

   # Get the 5 most similar documents based on the inferred vector
   sims = model.dv.most_similar([inferred_vector], topn=5)
   idx = [sims[i][0] for i in range(5)]

   # Print the most similar documents
   print('Test Document: «{}»\n'.format(' '.join(test_data)))
   print(u'SIMILAR DOCS PER MODEL %s:\n' % model)
   for label, index in [('1', 0), ('2', 1), ('3', 2), ('4', 3), ('5', 4)]:
       print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(tagged_data[int(sims[index][0])].words)))
   return idx

In [31]:
idx = get_most_similar_docs(test_data, model)

Test Document: «build 150 year six generation winemaking tradition winery trend toward lean style classic california buttercream aroma cut tart green apple .»

SIMILAR DOCS PER MODEL Doc2Vec<dm/m,d100,n5,w5,s0.001,t3>:

1 ('12', 0.6935006976127625): «build 150 year six generation winemaking tradition winery trend toward lean style classic california buttercream aroma cut tart green apple . pear barely ripe pineapple prove approachable distinctive . good everyday sip wine flavor range»

2 ('20306', 0.5462985038757324): «exotic yellow spice note meet lean lime pith light crisp nose old vine expression historic winery . meyer lemon rind juice show brightly palate grippy chalkiness complement rich lemon curd flavor .»

3 ('77168', 0.536984384059906): «come 100 year old vine fascinating wine show black cherry spearmint graphite pencil lead restrained nose . palate dry cherry tart skin plum red purple fruit keep nice sour tension throughout sip .»

4 ('12515', 0.5357843637466431): «light lea

In [ ]:
idx

['12', '95643', '2608', '95502', '46371']

In [32]:
reviews[['title', 'points', 'price', 'variety']].iloc[idx]

,title,points,price,variety
12,Mirassou 2012 Chardonnay (Central Coast),87,12.0,Chardonnay
20306,Chalone Vineyard 2014 Heritage Vines Estate Gr...,91,20.0,Pinot Blanc
77168,Lorenzi Estate 2011 Lopez Ranch Old VIne Zinfa...,88,38.0,Zinfandel
12515,Owen's Estate 1999 Chardonnay (South Eastern A...,84,15.0,Chardonnay
40657,Reichsgraf von Kesselstatt 2014 Kaseler Nies'c...,94,50.0,Riesling


In [37]:
tagged_data

[TaggedDocument(words=['firm', 'tannin', 'fill', 'juicy', 'red', 'berry', 'fruit', 'freshen', 'acidity', '.', 'ripe', 'fruity', 'wine', 'smooth', 'still', 'structure', '.', 'already', 'drinkable', 'although', 'certainly', 'well', '2016', '.'], tags=['0']),
 TaggedDocument(words=['tart', 'snappy', 'flavor', 'lime', 'flesh', 'rind', 'dominate', '.', 'green', 'pineapple', 'poke', 'crisp', 'acidity', 'underscore', 'flavor', '.', 'wine', 'stainless', 'steel', 'ferment', '.'], tags=['1']),
 TaggedDocument(words=['palate', 'bit', 'opulent', 'note', 'honey', 'drizzle', 'guava', 'mango', 'give', 'way', 'slightly', 'astringent', 'semidry', 'finish', '.', 'pineapple', 'rind', 'lemon', 'pith', 'orange', 'blossom', 'start', 'aroma', '.'], tags=['2']),
 TaggedDocument(words=['much', 'like', 'regular', 'bottling', '2012', 'come', 'across', 'rather', 'rough', 'tannic', 'rustic', 'earthy', 'herbal', 'characteristic', '.', 'nonetheless', 'think', 'pleasantly', 'unfussy', 'country', 'wine', 'good', 'comp